In [21]:
import pandas as pd
import numpy as np
import os
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn import set_config; set_config(display='diagram')
import torch
import matplotlib.pyplot as plt
from deepdow.benchmarks import OneOverN, Random, Benchmark
from deepdow.callbacks import EarlyStoppingCallback
from deepdow.data import InRAMDataset, RigidDataLoader, prepare_standard_scaler, Scale
from deepdow.data.synthetic import sin_single
from deepdow.experiments import Run
from deepdow.layers import SoftmaxAllocator
from deepdow.losses import MeanReturns, SharpeRatio, MaximumDrawdown
from deepdow.visualize import generate_metrics_table, generate_weights_table, plot_metrics, plot_weight_heatmap

In [22]:
# Step 1: Define a function to read CSV files and convert them to dataframes
# Get the current working directory (where your script is located)
current_directory = os.getcwd()

# Specify the path to the "raw_data" folder
raw_data_folder = os.path.join(current_directory, "raw_data")

# List all CSV files in the "raw_data" folder
csv_files = [os.path.join(raw_data_folder, file) for file in os.listdir(raw_data_folder) if file.endswith(".csv")]

# passing csv files in to a dataframe
dataframes = [pd.read_csv(file) for file in csv_files]

In [23]:
# Step 2: Define a function to read CSV files and convert them to dataframes
def read_csv_to_dataframe(file_path):
    df = pd.read_csv(file_path)
    return df

In [24]:
# Step 4: Read CSV files, preprocess data, and stack them into a 3D tensor
data = []
stock_names =[]

# print(type(csv_files))

for file in csv_files:
    # Read the CSV file into a DataFrame
    data_df = pd.read_csv(file)
    stock_name = file.split('/')[-1].split('.')[0]
    stock_names.append(stock_name)
    data_df["stock"] = stock_name
    data.append(data_df)

In [25]:
impute_columns = ["sma25", "sma100", "sma200", "rsi", "macd", "signal", "histogram"]

imputer = Pipeline(
    [
        ('imputer', ColumnTransformer(
            transformers=[
                ('impute', KNNImputer(n_neighbors=10), impute_columns),  # Apply imputation to specific columns
            ],
            remainder='passthrough'  # Keep the remaining columns
        ))
    ]
)

In [26]:
def df_and_column_transform(arr):
    df = pd.DataFrame(arr, columns=["sma25", "sma100", "sma200", "rsi", "macd", "signal", "histogram",\
    "Unnamed: 0", "open", "high","low", "close", "volume", "vwap", "timestamp", "transactions", "otc", "stock"])
    
    first_col = df.pop("sma25")
    df.insert(16, "sma25", first_col)

    sec_col = df.pop("sma100")
    df.insert(16, "sma100", sec_col)

    third_col = df.pop("sma200")
    df.insert(16, "sma200", third_col)

    fourth_col = df.pop("rsi")
    df.insert(16, "rsi", fourth_col)

    fifth_col = df.pop("macd")
    df.insert(16, "macd", fifth_col)
    
    sixth_col = df.pop("signal")
    df.insert(16, "signal", sixth_col)

    seventh_col = df.pop("histogram")
    df.insert(16, "histogram", seventh_col)
    
    return df

transform_pipe = make_pipeline(
    FunctionTransformer(df_and_column_transform)
)

preprocessor = preprocessor = Pipeline(
    [
        ("transformation", transform_pipe),
    ],
)
preprocessor


def timestamp_transform(df):
    df['date'] = pd.to_datetime(df['timestamp'], unit='ms').dt.date
    first_col = df.pop("date")
    df.insert(0, 'date', first_col)
    # df.set_index(keys='date', inplace=True)
    
    return df

time_pipe = make_pipeline(
    FunctionTransformer(timestamp_transform)
)

preprocessor = Pipeline(
    [
        ("transformation", transform_pipe),
        ("timestamp_convertor", time_pipe),
    ],
)
preprocessor

drop_columns = ["Unnamed: 0", 'timestamp', "transactions", "otc"]

def drop(df):
    unwanted_columns = drop_columns
    df = df.drop(columns=unwanted_columns)
    
    return df

drop_pipe = make_pipeline(
    FunctionTransformer(drop)
)

preprocessor = Pipeline(
    [
        ("transformation", transform_pipe),
        ("timestamp_convertor", time_pipe),
        ("unwanted_columns", drop_pipe),
    ],
)

In [27]:
sec_pipe = Pipeline(
    [
        ("imputer", imputer),
        ("preprocessor", preprocessor),
    ]
)

In [28]:
scaler_columns = ["open", "high", "low", "close", "volume", "vwap", "sma25", "sma100",
       "sma200", "rsi", "macd", "signal", "histogram"]

exclude = ["stock"]

# Create transformers for numeric and object columns
numeric_transformer = Pipeline(
    steps=[
        ('scaler', MinMaxScaler())
    ]
)

object_transformer = Pipeline(steps=[])

# Combine transformers using ColumnTransformer
sec_scaler = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, scaler_columns),
        # ('obj', object_transformer, exclude),
    ],
    remainder='passthrough'  # Keep the remaining columns
)

# Create a final pipeline
scaler = Pipeline(
    steps=[
        ('scaler', sec_scaler)
    ]
)


In [29]:
penul_pipe = Pipeline(
    [
        ("imputer", imputer),
        ("preprocessor", preprocessor),
        ("scaler", scaler)
    ]
)

In [30]:
def final_transformation(arr):
    cols = ["open", "high", "low", "close", "volume", "vwap", "sma25", "sma100", "sma200", "rsi", "macd",\
        "signal", "histogram", "date", "stock"]
    df = pd.DataFrame(arr, columns=cols)
    
    first_col = df.pop("date")
    df.insert(0, "date", first_col)
    
    df.set_index(keys='date', inplace=True)
    
    return df

final_transformation_pipe = make_pipeline(
    FunctionTransformer(final_transformation)
)

final_processing = Pipeline(
    [
        ("final_transformation", final_transformation_pipe)
    ]
)

In [31]:
final_pipe = Pipeline(
    [
        ("imputer", imputer),
        ("preprocessor", preprocessor),
        ("scaler", scaler),
        ("transformation", final_processing)
    ]
)

In [32]:
final_pipe.fit_transform(data[0])

,open,high,low,close,volume,vwap,sma25,sma100,sma200,rsi,macd,signal,histogram,stock
date,,,,,,,,,,,,,,
2018-08-31,0.057931,0.05447,0.06204,0.058653,0.168732,0.0578,0.409796,0.465189,0.525993,0.455593,0.491663,0.540682,0.451672,AMD
2018-09-04,0.0629,0.073962,0.068091,0.078549,0.611726,0.073274,0.409796,0.465189,0.525993,0.455593,0.491663,0.540682,0.451672,AMD
2018-09-05,0.0887,0.08639,0.077176,0.081647,0.837191,0.081194,0.409796,0.465189,0.525993,0.455593,0.491663,0.540682,0.451672,AMD
2018-09-06,0.079918,0.077154,0.079674,0.077034,0.442654,0.077922,0.409796,0.465189,0.525993,0.455593,0.491663,0.540682,0.451672,AMD
2018-09-07,0.072022,0.075048,0.07689,0.073868,0.371007,0.076237,0.409796,0.465189,0.525993,0.455593,0.491663,0.540682,0.451672,AMD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-24,0.64452,0.641267,0.605704,0.58619,0.321712,0.60702,0.705034,0.779901,0.724236,0.239357,0.360995,0.451613,0.330443,AMD
2023-08-25,0.577161,0.590194,0.596487,0.589288,0.202084,0.595187,0.702453,0.780479,0.726297,0.250737,0.347151,0.433955,0.320592,AMD
2023-08-28,0.592852,0.589854,0.605871,0.591766,0.13457,0.599475,0.699245,0.78139,0.728588,0.260325,0.339155,0.434939,0.310832,AMD


In [33]:
dataframes = []

for df in data:
    preprocessed_df = final_pipe.fit_transform(df)
    
    dataframes.append(preprocessed_df)

In [34]:
filtered_dataframes = []

for df in dataframes:
    if df.shape == (1257, 14):
        filtered_dataframes.append(df)

In [35]:
unequal_shape = []

for df in dataframes:
    if df.shape != (1257, 14):
        unequal_shape.append(df)

In [36]:
# Check
num_dataframes = len(dataframes)
print("Number of dataframes:", num_dataframes)

unequal = len(unequal_shape)
print(f"Length of unequal dataframe: {unequal}")

f = len(filtered_dataframes)
print(f"Length of filtered dataframe: {f}")

Number of dataframes: 100
Length of unequal dataframe: 3
Length of filtered dataframe: 97


In [37]:
named_dfs = [(df.iloc[:, :-1], df.iloc[:, -1].iloc[0]) for df in filtered_dataframes]
len(named_dfs)

97

(                open      high       low     close    volume      vwap  \
 date                                                                     
 2018-08-31  0.057931   0.05447   0.06204  0.058653  0.168732    0.0578   
 2018-09-04    0.0629  0.073962  0.068091  0.078549  0.611726  0.073274   
 2018-09-05    0.0887   0.08639  0.077176  0.081647  0.837191  0.081194   
 2018-09-06  0.079918  0.077154  0.079674  0.077034  0.442654  0.077922   
 2018-09-07  0.072022  0.075048   0.07689  0.073868  0.371007  0.076237   
 ...              ...       ...       ...       ...       ...       ...   
 2023-08-24   0.64452  0.641267  0.605704   0.58619  0.321712   0.60702   
 2023-08-25  0.577161  0.590194  0.596487  0.589288  0.202084  0.595187   
 2023-08-28  0.592852  0.589854  0.605871  0.591766   0.13457  0.599475   
 2023-08-29  0.583594  0.606833  0.612265  0.614553  0.118787  0.619034   
 2023-08-30  0.606004  0.608123  0.631613  0.618753  0.033785  0.623717   
 
                sma25  

In [44]:
layered_dfs = []
for i in range(len(named_dfs)):
    df = named_dfs[i][0]
    df.columns = pd.MultiIndex.from_product([[named_dfs[i][1]], df.columns],
                                     names=['share', 'feature'])
    layered_dfs.append(df)

In [45]:
combined_df = pd.concat([df for df, _ in named_dfs], axis=1)

In [46]:
combined_df

share            AMD                                                    \
feature         open      high       low     close    volume      vwap   
date                                                                     
2018-08-31  0.057931   0.05447   0.06204  0.058653  0.168732    0.0578   
2018-09-04    0.0629  0.073962  0.068091  0.078549  0.611726  0.073274   
2018-09-05    0.0887   0.08639  0.077176  0.081647  0.837191  0.081194   
2018-09-06  0.079918  0.077154  0.079674  0.077034  0.442654  0.077922   
2018-09-07  0.072022  0.075048   0.07689  0.073868  0.371007  0.076237   
...              ...       ...       ...       ...       ...       ...   
2023-08-24   0.64452  0.641267  0.605704   0.58619  0.321712   0.60702   
2023-08-25  0.577161  0.590194  0.596487  0.589288  0.202084  0.595187   
2023-08-28  0.592852  0.589854  0.605871  0.591766   0.13457  0.599475   
2023-08-29  0.583594  0.606833  0.612265  0.614553  0.118787  0.619034   
2023-08-30  0.606004  0.608123  0.631613  0.618753  0.033785  0.623717   

share                                               ...      NVDA            \
feature        sma25    sma100    sma200       rsi  ...     close    volume   
date                                                ...                       
2018-08-31  0.409796  0.465189  0.525993  0.455593  ...  0.082811  0.086469   
2018-09-04  0.409796  0.465189  0.525993  0.455593  ...  0.084439  0.121727   
2018-09-05  0.409796  0.465189  0.525993  0.455593  ...  0.081593  0.122335   
2018-09-06  0.409796  0.465189  0.525993  0.455593  ...   0.07852  0.093184   
2018-09-07  0.409796  0.465189  0.525993  0.455593  ...  0.078056  0.081841   
...              ...       ...       ...       ...  ...       ...       ...   
2023-08-24  0.705034  0.779901  0.724236  0.239357  ...  0.948577  0.438343   
2023-08-25  0.702453  0.780479  0.726297  0.250737  ...  0.923885  0.342813   
2023-08-28  0.699245   0.78139  0.728588  0.260325  ...  0.941504  0.243248   
2023-08-29  0.697958  0.782609  0.730598  0.345629  ...  0.983535   0.25004   
2023-08-30  0.696547   0.78361  0.732431  0.360724  ...       1.0  0.106805   

share                                                                   \
feature         vwap     sma25    sma100    sma200       rsi      macd   
date                                                                     
2018-08-31  0.083028  0.272274  0.299715  0.398683  0.508324   0.40558   
2018-09-04  0.084647  0.272274  0.299715  0.398683  0.508324   0.40558   
2018-09-05  0.082151  0.272274  0.299715  0.398683  0.508324   0.40558   
2018-09-06  0.079354  0.272274  0.299715  0.398683  0.508324   0.40558   
2018-09-07  0.079129  0.272274  0.299715  0.398683  0.508324   0.40558   
...              ...       ...       ...       ...       ...       ...   
2023-08-24  0.983474   0.99081  0.976225   0.97418  0.604025   0.49011   
2023-08-25  0.933556  0.992164  0.981592  0.980385  0.529309  0.495802   
2023-08-28  0.934738  0.993278  0.987361  0.986914  0.567909  0.511908   
2023-08-29  0.982476  0.996487  0.993649  0.993439    0.6482  0.554232   
2023-08-30       1.0       1.0       1.0       1.0  0.675834  0.597509   

share                           
feature       signal histogram  
date                            
2018-08-31  0.413179  0.387224  
2018-09-04  0.413179  0.387224  
2018-09-05  0.413179  0.387224  
2018-09-06  0.413179  0.387224  
2018-09-07  0.413179  0.387224  
...              ...       ...  
2023-08-24  0.613234  0.401761  
2023-08-25  0.585691   0.41874  
2023-08-28  0.587046  0.435851  
2023-08-29  0.647016  0.458811  
2023-08-30  0.697134  0.486658  

[1257 rows x 1261 columns]

In [47]:
port_df = combined_df[['AAPL','GOOG','MSFT','AMZN']]

In [48]:
idx = pd.IndexSlice
test_df = port_df.loc[idx[:], idx[:, 'close']]

In [49]:
test_df.columns = test_df.columns.remove_unused_levels()

In [50]:
test_df

share,AAPL,GOOG,MSFT,AMZN
feature,close,close,close,close
date,,,,
2018-08-31,0.132751,0.118731,0.068586,0.280111
2018-09-04,0.133885,0.108334,0.06625,0.291336
2018-09-05,0.13157,0.103172,0.054115,0.272617
2018-09-06,0.125713,0.095792,0.055057,0.257325
2018-09-07,0.122916,0.092548,0.05306,0.254711
...,...,...,...,...
2023-08-24,0.875266,0.800889,0.85107,0.541515
2023-08-25,0.889125,0.803539,0.862413,0.553411
